In [182]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm;
import imageio
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
from itertools import zip_longest

In [177]:
TEST_DF = pd.read_csv('data/test.csv')

In [178]:
def build_image_names(image_id: str) -> list:
    # mt is the mitchondria
    mt = f'pcl.hpa/data/HPA2020/JPEGImages/{image_id}_red.png'    
    # er is the endoplasmic reticulum
    er = f'pcl.hpa/data/HPA2020/JPEGImages/{image_id}_yellow.png'    
    # nu is the nuclei
    nu = f'pcl.hpa/data/HPA2020/JPEGImages/{image_id}_blue.png'    
    return mt, er, nu

NUC_MODEL = 'data/dpn_unet_nuclei_v1.pth'
CELL_MODEL = 'data/dpn_unet_cell_3ch_v1.pth'

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device='cpu',
    padding=True,
    multi_channel_model=True
)

In [ ]:
for i in range(8, len(TEST_DF), 8):
    
    RED = []
    YEL = []
    BLU = []

    for im in list(TEST_DF['ID'])[i - 8: i]:
        mt, er, nu = build_image_names(im)
        RED.append(mt)
        YEL.append(er)
        BLU.append(nu)

    batch = [RED, YEL, BLU]
    
    cell_segmentations = segmentator.pred_cells(batch)
    nuc_segmentations = segmentator.pred_nuclei(batch[2])


    n = len(cell_segmentations)

    for i, n, c in zip(range(n), nuc_segmentations, cell_segmentations):
        fn = RED[i].split('/')[-1].split('_')[0]
        nuc_mask, cell_mask = label_cell(n, c)
        np.savez(f'masks/hpa_nuclei_mask_test/{fn}', nuc_mask)
        np.savez(f'masks/hpa_cell_mask_test/{fn}', cell_mask)